# Building a spam detection filter using python and spark!
### out data set consist of volunteered text meassages form a study in singapore and some spam texts from a UK reporting site.

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
data = spark.read.csv('/home/shyam/NiceSoftwareSolutions/MachineLearning/NLP/SMSSpamCollection.tsv',
                      inferSchema=True,sep='\t')

In [4]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [5]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [6]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import length

In [8]:
data = data.withColumn('length',length(data['text']))

In [9]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [10]:
from pyspark.ml.feature import (Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer)

In [11]:
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text',outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')
idf = IDF(inputCol='c_vec',outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [12]:
from pyspark.ml.feature import VectorAssembler

In [13]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [14]:
from pyspark.ml.classification import NaiveBayes

In [15]:
nb = NaiveBayes()

In [16]:
from pyspark.ml import Pipeline

In [17]:
data_prep_line = Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove,count_vec,idf,clean_up])

In [18]:
cleaner = data_prep_line.fit(data)

In [19]:
clean_data = cleaner.transform(data)

In [20]:
clean_data = clean_data.select('label','features')

In [21]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [22]:
training, test = clean_data.randomSplit([0.7,0.3])

In [23]:
spam_detector = nb.fit(training)

In [24]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [25]:
test_results  = spam_detector.transform(test)

In [26]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,13,...|[-608.12650891399...|[1.0,8.6924230326...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-575.44232729212...|[0.99999999999897...|       0.0|
|  0.0|(13424,[0,1,4,50,...|[-835.94108449383...|[1.0,2.9595067193...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-1002.8267624118...|[1.0,5.3697878872...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-884.80177615364...|[1.0,1.6020067744...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1206.0222440356...|[1.0,1.5158651509...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-655.15252228808...|[1.0,1.7958909575...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-541.48926686942...|[1.0,7.7229168561...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-696.20274028632...|[1.0,5.2501504584...|       0.0|
|  0.0|(13424,[0

In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [28]:
acc_eval = MulticlassClassificationEvaluator()

In [29]:
acc = acc_eval.evaluate(test_results)

In [30]:
print("Accuracy of Naive Bayes Model")
print(acc)

Accuracy of Naive Bayes Model
0.9134577224776506
